In [95]:
#Libraries
import pypyodbc
import pandas as pd
#Conn and credentials
conn_str = 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:servidortcprueba.database.windows.net,1433;Database=db_campechanos;Uid=adminraul;Pwd=5ccu9$j$XzhNMMF;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
connection = pypyodbc.connect(conn_str)
cursor = connection.cursor()

#Training dataset
query = "SELECT * FROM train"
cursor.execute(query)
rows = cursor.fetchall()
df_train = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_train.columns = columns

#Test dataset, dont have sales price
query = "SELECT * FROM test"
cursor.execute(query)
rows = cursor.fetchall()
df_test = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_test.columns = columns

#Sample dataset, id and sales price
query = "SELECT * FROM sample"
cursor.execute(query)
rows = cursor.fetchall()
df_sample = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_sample.columns = columns

In [96]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
df_test=df_test.drop('na_mszoning',axis=1)
df_test= df_test.drop('na_saletype',axis=1)
columns_in_test = df_test.columns

# Filtra las columnas en df_train que también están en df_test
df_train_filtered = df_train[columns_in_test]
X_train = df_train.drop('saleprice', axis=1)  # Ajusta 'target_column' a tu columna objetivo real
y_train = df_train['saleprice']
X_train = X_train.drop('id',axis=1)

X_train=X_train.drop('is_house_25fstory',axis=1)
columns_in_train = X_train.columns

# Filtra las columnas en df_test que también están en df_train
df_test_filtered = df_test[columns_in_train]
X_test = df_test_filtered
y_test = df_sample['saleprice']

## Model 1 without changes

In [97]:
for col in X_test.columns:
    print(col)

lotfrontage
lotarea
overallqual
overallcond
yearbuilt
yearremodadd
masvnrarea
bsmtfinsf1
bsmtfinsf2
bsmtunfsf
totalbsmtsf
_1stflrsf
_2ndflrsf
lowqualfinsf
grlivarea
bsmtfullbath
bsmthalfbath
fullbath
halfbath
bedroomabvgr
kitchenabvgr
totrmsabvgrd
fireplaces
garageyrblt
garagecars
garagearea
wooddecksf
openporchsf
enclosedporch
_3ssnporch
screenporch
poolarea
miscval
mosold
yrsold
is_zone_lres
is_zone_mres
is_zone_com
is_zone_float
is_zone_hres
is_land_lvl
is_land_bk
is_land_low
is__land_hi
is_lot_in
is_lot_fr2
is_lot_cor
is_lot_culdsac
is_fr3
is_bldg_singlef
is_bldg_twof
is_bldg_du
is_bldg_townend
is_bldg_townins
is_house_2story
is_house_1story
is_house_15fstory
is_house_15ustory
is_house_splitfr
is_house_splitlvl
is_house_25ustory
is_roof_gable
is_roof_hip
is_roof_gambrel
is_roof_mansard
is_roof_flat
is_roof_shed
is_masn_brkface
is_masn_none
is_masn_stone
is_masn_brk
is_masn_na
is_foun_pconc
is_foun_cblock
is_foun_brktil
is_foun_wood
is_foun_slab
is_foun_stone
is_garage_attchd
is_gar

In [104]:
len(X_test.columns)

128

In [105]:
import numpy as np
X_train= X_train.replace(np.nan,0)
X_test= X_test.replace(np.nan,0)

In [107]:
X_test.to_csv('Test_Submits.csv', index=False)

In [112]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Crear un modelo de bosque aleatorio
model = RandomForestRegressor(random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")


Error cuadrático medio (MSE): 4943187587.234059
Coeficiente de determinación (R^2): -17.12900632190139


In [116]:
from sklearn.metrics import mean_squared_error, r2_score
# Calcular el MSE (Error Cuadrático Medio)
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

# Calcular el RMSE (Raíz del Error Cuadrático Medio)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Calcular el R^2 (Coeficiente de Determinación)
r2 = r2_score(y_test, y_pred)
print(f"R^2: {r2}")

MSE: 4943187587.234059
RMSE: 70307.80601920429
R^2: -17.12900632190139


In [100]:
predictions_df = pd.DataFrame({'Id': df_sample['id'], 'SalePrice': y_pred})
predictions_df.to_csv('Apredicciones.csv', index=False)

# Imprime las primeras filas del archivo CSV
print(predictions_df.head())

     Id  SalePrice
0  1461  128730.08
1  1462  159078.40
2  1463  184869.09
3  1464  182197.30
4  1465  195641.56


In [101]:
import joblib
joblib.dump(model,'arbol.joblib')

['arbol.joblib']

In [102]:
loaded_model = joblib.load('arbol.joblib')

In [85]:
X_new = df_test.loc[1].values


In [86]:
predictions = loaded_model.predict(X_new.reshape(1, -1))
print(predictions)

[152303.49]


c:\Users\rauli\anaconda3\envs\IA\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
